In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [2]:
PATH_TO_DATA = '../ru_kw_eval_datasets-master/data'
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]
data = pd.concat([pd.read_json(file, lines=True) for file in files][:1], axis=0, ignore_index=True)

In [85]:
data.to_csv('data_text.csv')

In [3]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

In [4]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

def normalize_kw(text):
    text = ' '.join(text)
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [5]:
data['norm_content'] = data.content.apply(normalize)
data['norm_kw'] = data.keywords.apply(normalize_kw)

In [7]:
data['norm_content'] = data.norm_content.apply(lambda x:' '.join(x))

In [50]:
from rake_nltk import Metric, Rake

In [6]:
data.head()

,content,keywords,summary,title,url,norm_content,norm_kw
0,"Многие интересуются, зачем нужна «Яблоку» моло...","[яблоко, молодежь, молодежное яблоко]",,"""Молодежное ""Яблоко"": оппозиционная деятельнос...",http://www.ng.ru/ng_politics/2017-04-18/11_697...,"[многие, интересоваться, нужный, яблоко, молод...","[яблоко, молодёжь, молодёжный, яблоко]"
1,Вчера «Газпром» снизил верхнюю планку прогноза...,"[газпром, газ]",,"""Газпрома"" на всех не хватит",http://www.ng.ru/economics/2008-04-03/1_gazpro...,"[вчера, газпром, снизить, верхний, планка, про...","[газпром, газ]"
2,Долголетний труд Евгения Витковского на ниве п...,"[франсуа рабле, сервантес, шекспир, конан дойл...","Евгений Витковский о том, как Босх протягивает...",Бесконечная партия в четырехмерные шахматы,http://www.ng.ru/person/2018-03-22/10_927_vitk...,"[долголетний, труд, евгений, витковский, нива,...","[франсуа, рабле, сервантес, шекспир, конана, д..."
3,В Ленинском районном суде продолжаются слушани...,"[владивосток, суд, ким, футина, выборы, боевое...",Фигурантке уголовного дела о фальсификации выб...,"Экс-депутат, осужденная за фальсификацию выбор...",http://www.ng.ru/regions/2018-01-10/100_vladiv...,"[ленинский, районный, суд, продолжаться, слуша...","[владивосток, суд, ким, футиный, выбор, боевой..."
4,В 2012 году российская столица резко увеличила...,"[новая москва, подмосковье, благоустройство, т...",Лучшие проекты благоустройства общественных пр...,Новая Москва останется территорией экологическ...,http://www.ng.ru/ng_stolitsa/2017-11-10/10_711...,"[2012, год, российский, столица, резко, увелич...","[новый, москва, подмосковье, благоустройство, ..."


In [9]:
from gensim.corpora import *
texts = [data.norm_content.iloc[i].split() for i in range(len(data))]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [10]:
%%time
from gensim.models import  *
from gensim import similarities

tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

index = similarities.MatrixSimilarity(corpus_tfidf)
sims = index[corpus_tfidf]

CPU times: user 20.8 s, sys: 984 ms, total: 21.8 s
Wall time: 35.7 s


In [11]:
%%time
lsi = lsimodel.LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=30)

CPU times: user 23.1 s, sys: 4.93 s, total: 28.1 s
Wall time: 12.8 s


In [12]:
lsi.show_topics(5)

[(0,
  '0.082*"россия" + 0.077*"страна" + 0.076*"украина" + 0.075*"сша" + 0.071*"партия" + 0.071*"путин" + 0.070*"сирия" + 0.068*"военный" + 0.067*"власть" + 0.066*"президент"'),
 (1,
  '-0.343*"гонка" + -0.293*"пилот" + -0.249*"команда" + -0.171*"трасса" + -0.134*"заезд" + -0.111*"ferrari" + -0.109*"машина" + -0.105*"хэмилтон" + -0.103*"нико" + -0.103*"подиум"'),
 (2,
  '0.175*"гонка" + 0.162*"сирия" + 0.158*"пилот" + 0.146*"газа" + -0.143*"книга" + 0.123*"сирийский" + 0.120*"команда" + -0.108*"поэт" + 0.108*"газопровод" + -0.100*"фильм"'),
 (3,
  '-0.240*"сирия" + -0.195*"сирийский" + 0.164*"газа" + -0.144*"дамаск" + -0.120*"военный" + 0.107*"миллиард" + 0.100*"куб" + 0.096*"компания" + -0.096*"вашингтон" + 0.089*"метр"'),
 (4,
  '-0.189*"сирия" + 0.162*"партия" + -0.155*"сирийский" + -0.141*"книга" + 0.136*"украина" + -0.118*"поэт" + -0.112*"стих" + 0.110*"выбор" + -0.109*"дамаск" + 0.109*"путин"')]

In [74]:
def rank(text):
    # To use it with a specific language supported by nltk.
    r = Rake(language='ru', stopwords=stops,
    punctuations=punct, ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO, min_length=1, max_length=4)
    r.extract_keywords_from_text(text)
    kw = r.get_ranked_phrases()
    return list(set(' '.join(kw).split(' ')))

In [75]:
data['pred_kw'] = data.norm_content.apply(rank)

In [76]:
evaluate(data['keywords'], data['pred_kw'])

Precision -  0.02
Recall -  0.04
F1 -  0.03
Jaccard -  0.01


In [65]:
import yake

In [80]:
import numpy as np
simple_kwextractor = yake.KeywordExtractor(lan="ru", n=1, dedupLim=0.8, windowsSize=2, top=20)
kws=[]
for i in range(data.shape[0]):
    keywords = simple_kwextractor.extract_keywords(data.norm_content[i])
    kw = [j for i,j in keywords]
    kws.append(kw)
evaluate(data.norm_kw, np.array(kws))

Precision -  0.14
Recall -  0.44
F1 -  0.2
Jaccard -  0.12
